# Web Scraping - Asignment 3
By Mónica Atiaga

Batch - DSNB1222

Instructions: 
* All questions are compulsory. 
* In each of the questions you have to automate the process. You do not have to click on any button, click any 
clickable element, enter keywords in search boxes manually. Each process has to be performed via coding. 
* Q1 and Q2 are connected questions i.e. after attempting Q1 proceed to Q2. Do not write whole code from 
beginning for Q2. 
* You may use any web scraping library and tools. 
* The question can be attempted in various ways; the correctness of question depends on the output. 
* If you encounter any Null values during scraping, you may replace it by hyphen. 



In [2]:
# Importing libraries 
import selenium
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

import time

Exercise: 
1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars. 


In [49]:
def search_product(product, xpath_input, xpath_btn):
    try:
        input = driver.find_element(By.XPATH, xpath_input) 
        input.send_keys(product)
        print(f"Search for {product}")

        WebDriverWait(driver, 10)

        btn = driver.find_element(By.XPATH, xpath_btn) 
        btn.click()
        print("Click successful")

    except NoSuchElementException as e:
        print(f"An element wasn't found. {e.msg}")


In [65]:
# Asking to the user for a word to search
product = input("Please, enter the product do you want to search?")
print(f"We are going to searching for: {product}")

# Open driver 
driver = webdriver.Chrome()

url = "https://www.amazon.in/"
driver.get(url)

wait_time = 3
time.sleep(wait_time)

xpath_input = '//input[@id="twotabsearchtextbox"]'
xpath_btn = '//span[@id="nav-search-submit-text"]'

search_product(product, xpath_input, xpath_btn)

Please, enter the product do you want to search?guitar
We are going to searching for: guitar
Search for guitar
Click successful


2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. 
    Details to be scraped are: 
        "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", 
        "Availability", and “Product URL”. 
    In case, if any of the details are missing for any of the product then replace it by “-“. 

In [99]:
def go_next_page(driver, xpath):
    ''' Go to the next page'''
        
    next_btn = driver.find_element(By.XPATH, xpath)
    print(f" Going to {next_btn.text}")
    
    # scroll until button be visible
    actions = ActionChains(driver)
    actions.move_to_element(next_btn).perform()

    # Wait until the button be visible (loading compleated)
    wait = WebDriverWait(driver, 15)
    wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))

    # Click the button once it is clickable
    next_btn.click()
    
def scrape_product(driver, **kwargs):
    '''Retrieve all the attributes sent in kwargs.
       Return: a Data frame with the data.'''
    data = {}
    for key, xpath in kwargs.items():
        try:
            tag = driver.find_element(By.XPATH, xpath)
            data[key] = [tag.text]
        except NoSuchElementException as e:
            print(f"Exception: {e.msg}")
            data['key'] = ['-']
    return pd.DataFrame(data)

In [66]:
# Getting the links for each searched product
pages = 3
urls = []

for i in range(pages):
    # Retrieving urls
    xpath = '//h2[@class="a-size-mini a-spacing-none a-color-base s-line-clamp-4"]//a'
    link_tags = driver.find_elements(By.XPATH, xpath)
    links = [el.get_attribute('href') for el in link_tags]
    print(f"Retrieved urls: {len(links)} in page {i+1}")
    urls = urls + links
    
    if (i < pages - 1):
        # Going to Next page
        xpath_nextbtn = '//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]'
        go_next_page(driver, xpath_nextbtn)

    wait_time=5
    time.sleep(wait_time)

print(f"Total urls: {len(urls)}")

Retrieved urls: 60 in page 1
 Going to Next
Retrieved urls: 60 in page 2
 Going to Next
Retrieved urls: 60 in page 3
 Going to Next
Total urls: 180


In [102]:
attributes = {
    'Brand': '//table[@class="a-normal a-spacing-micro"]/tbody/tr[1]/td[@class="a-span9"]/span',
    'Name of the product': '//span[@id="productTitle"]',
    'Price': '//span[@class="a-price aok-align-center reinventPricePriceToPayMargin priceToPay"]//span//span[@class="a-price-whole"]',
    'Return/Exchange': '//div[@data-name="RETURNS_POLICY"]',
    'Expected Delivery': '//div[@id="mir-layout-DELIVERY_BLOCK"]/div/span/span',
    'Availability': '//div[@id="availability"]'
}

# Initializing dataframe
df = pd.DataFrame(columns=[*attributes.keys()])

for url in urls:
    # url = urls[0]
    driver.get(url)

    wait_time = 3
    time.sleep(wait_time)

    # Scrape the data in each url
    df2 = scrape_product(driver, **attributes)
    df = pd.concat([df, df2], ignore_index=True)

display(df)
    

Exception: no such element: Unable to locate element: {"method":"xpath","selector":"//span[@class="a-price aok-align-center reinventPricePriceToPayMargin priceToPay"]//span//span[@class="a-price-whole"]"}
  (Session info: chrome=115.0.5790.110); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Exception: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@id="mir-layout-DELIVERY_BLOCK"]/div/span/span"}
  (Session info: chrome=115.0.5790.110); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Exception: no such element: Unable to locate element: {"method":"xpath","selector":"//span[@class="a-price aok-align-center reinventPricePriceToPayMargin priceToPay"]//span//span[@class="a-price-whole"]"}
  (Session info: chrome=115.0.5790.110); For documentation on this error, pleas

,Brand,Name of the product,Price,Return/Exchange,Expected Delivery,Availability,key
0,Kadence,Kadence A281 Professional Acoustic Rosewood gu...,"6,499",7 days Replacement,"Friday, 11 August",In stock,NaN
1,Kadence,"Kadence rosewood Guitar Frontier Series, Elect...","4,999",7 days Replacement,"Friday, 11 August",In stock,NaN
2,Intern,Intern INT-38C Acoustic Guitar Kit (Red),"2,099",7 days Replacement,"Wednesday, 9 August",In stock,NaN
3,Intern,Intern INT-38C-NT-G 38 Incheses lindenwood Rig...,"2,239",7 days Replacement,"Wednesday, 9 August",In stock,NaN
4,JUAREZ,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...","1,799",7 days Replacement,"Wednesday, 9 August",In stock,NaN
...,...,...,...,...,...,...,...
175,Fender,"Fender CD-60 0961545032, 6-Strings Acoustic Gu...","13,279",7 days Replacement,"Friday, 11 August",Only 2 left in stock,NaN
176,Gibz's guitar,"Gibz's Guitar FT-G41 Frontier Series 41"" Jumbo...","6,999",7 days Replacement,"Saturday, 12 August",Only 2 left in stock,NaN
177,Yellow and Green,NutCor Portable Electronic Musical Mini Guitar...,239,10 days Returnable,"Saturday, 12 August",In stock,NaN
178,Guitar Bro,GUITAR BRO - COMBO (Black Acoustic Rose Wood G...,"8,999",7 days Replacement,11 - 14 August,In stock,NaN


In [114]:
# Cleaning driver
driver.quit()

df.drop('key', axis=1, inplace=True)

# Assigning the URLs to the data frame
df['Product URL'] = urls
display(df)

# Saving the data frame
df.to_csv('datasets/q2_guitars.csv')

,Brand,Name of the product,Price,Return/Exchange,Expected Delivery,Availability,Product URL
0,Kadence,Kadence A281 Professional Acoustic Rosewood gu...,"6,499",7 days Replacement,"Friday, 11 August",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Kadence,"Kadence rosewood Guitar Frontier Series, Elect...","4,999",7 days Replacement,"Friday, 11 August",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Intern,Intern INT-38C Acoustic Guitar Kit (Red),"2,099",7 days Replacement,"Wednesday, 9 August",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,Intern,Intern INT-38C-NT-G 38 Incheses lindenwood Rig...,"2,239",7 days Replacement,"Wednesday, 9 August",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,JUAREZ,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...","1,799",7 days Replacement,"Wednesday, 9 August",In stock,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...
175,Fender,"Fender CD-60 0961545032, 6-Strings Acoustic Gu...","13,279",7 days Replacement,"Friday, 11 August",Only 2 left in stock,https://www.amazon.in/Fender-0961545032-6-Stri...
176,Gibz's guitar,"Gibz's Guitar FT-G41 Frontier Series 41"" Jumbo...","6,999",7 days Replacement,"Saturday, 12 August",Only 2 left in stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
177,Yellow and Green,NutCor Portable Electronic Musical Mini Guitar...,239,10 days Returnable,"Saturday, 12 August",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
178,Guitar Bro,GUITAR BRO - COMBO (Black Acoustic Rose Wood G...,"8,999",7 days Replacement,11 - 14 August,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...


In [120]:
# Checking not found values
df.describe()

,Brand,Name of the product,Price,Return/Exchange,Expected Delivery,Availability,Product URL
count,180,180,174,180,174,180,180
unique,30,165,128,4,11,5,180
top,JUAREZ,Kadence A281 Professional Acoustic Rosewood gu...,"2,499",7 days Replacement,"Thursday, 10 August",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
freq,27,2,6,170,38,153,1


In [126]:
print(df['Price'].unique())
print(df['Expected Delivery'].unique())

['6,499' '4,999' '2,099' '2,239' '1,799' '7,099' '1,849' '2,649' '2,399'
 '12,500' '4,979' '1,539' '9,490' '2,019' '4,609' '2,499' '11,999'
 '13,119' '2,229' '2,999' '7,249' '5,910' '6,999' '2,089' '1,919' '8,490'
 '2,939' '1,999' '2,699' '1,792' '2,039' '2,349' '6,899' '2,299' '10,459'
 '14,850' '2,869' '7,989' '5,962' '2,799' '2,489' '10,769' '8,549' '1,969'
 '2,379' '4,801' '4,993' '1,290' '4,635' '4,349' '2,655' '4,029' '13,490'
 '3,089' '2,249' '9,290' '3,999' '3,899' '4,350' '7,399' '14,999' nan
 '12,490' '9,690' '4,590' '2,695' '4,290' '8,990' '1,979' '9,629' '6,429'
 '7,928' '3,379' '10,989' '3,358' '2,609' '2,759' '14,190' '27,979'
 '2,459' '5,499' '2,149' '3,990' '5,589' '23,199' '15,469' '18,659'
 '25,500' '3,290' '44,484' '895' '6,934' '2,895' '22,489' '13,790' '7,131'
 '10,875' '10,499' '10,941' '5,699' '7,599' '5,999' '8,499' '12,699'
 '2,395' '3,789' '7,999' '4,076' '18,091' '16,399' '14,990' '12,199'
 '11,489' '5,899' '7,400' '12,453' '13,639' '4,490' '12,499' '15,499'


In [127]:
registros_con_nan = df[df.isna().any(axis=1)]
registros_con_nan

,Brand,Name of the product,Price,Return/Exchange,Expected Delivery,Availability,Product URL
75,Fender,Fender Acoustic Guitar Auditorium Cutaway Elec...,NaN,Returns Policy,NaN,,https://www.amazon.in/Fender-FA-345CE-Auditori...
84,Medellin,Medellin M38 carbon fiber body 38 Incheses Aco...,NaN,Returns Policy,NaN,,https://www.amazon.in/Medellin-Acoustic-Guitar...
136,JUAREZ,JUAREZ Acoustic Guitar 3xE 1st String & 2 Pick...,NaN,Returns Policy,NaN,,https://www.amazon.in/JUAREZ-Acoustic-Guitar-S...
148,YAMAHA,"YAMAHA Pacifica012 Electric Guitar, Red Metallic",NaN,Returns Policy,NaN,Currently unavailable.\nWe don't know when or ...,https://www.amazon.in/Yamaha-PAC012-Electric-G...
149,YAMAHA,Yamaha 22.8 Inches Merantiwood Classical Guita...,NaN,Returns Policy,NaN,,https://www.amazon.in/Yamaha-Classical-Guitar-...
151,C M enterprise,C M enterprise 4 String Acoustic Décor Guitar ...,NaN,Returns Policy,NaN,,https://www.amazon.in/enterprise-Acoustic-D%C3...


In [130]:
# Filling the NaN with '-'
df_filled = df.fillna('-')

# Checking there is no NaN
registros_con_nan = df_filled[df_filled.isna().any(axis=1)]
registros_con_nan

,Brand,Name of the product,Price,Return/Exchange,Expected Delivery,Availability,Product URL


In [129]:
# Saving the data frame
df_filled.to_csv('datasets/q2_guitars.csv')

3. Write a python program to access the search bar and search button on images.google.com and scrape 10 
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’. 


In [170]:
# Importing libraries 
import selenium
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

import time
import os
import requests

In [207]:
def download_images(image_urls, keyword, n_reg):
    '''Create a directory and download n_reg images from the list image_urls'''
    # Create a directory to save images
    os.makedirs(f"q3_{keyword}", exist_ok=True)

    # Checking the valid images
    if len(image_urls) < n_reg:
        n_reg = len(image_urls)

    # Download the images
    for i, url in enumerate(image_urls[:n_reg]):
    # for i in range(n_reg):
        print(f"Downloading image of {keyword} {i+1}/{n_reg}")
        response = requests.get(url)
        with open(os.path.join(f"q3_{keyword}", f"{keyword}_{i+1}.jpg"), 'wb') as f:
            f.write(response.content)


#### Web scraping - Begin 

In [214]:
keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']
url = 'http://images.google.com/'

xpath_input = '//textarea[@id="APjFqb"]'
xpath_btn = '//button[@aria-label="Buscar con Google"]'
xpath_img = '//img[@class="rg_i Q4LuWd"]'
n_reg = 10

# Set up the WebDriver 
driver = webdriver.Chrome()

for keyword in keywords:
    driver.get(url)

    wait_time = 2
    time.sleep(wait_time)
    
    print('\n','*'*40)
    search_product(keyword, xpath_input, xpath_btn)

    # Scroll down to load more images
    for _ in range(3):  # Scroll 3 times to load more images
        driver.execute_script("window.scrollBy(0, 1000)")
        time.sleep(1)

    try:
        # Scrape the image URLs
        image_urls = []
        tag_urls = driver.find_elements(By.XPATH, xpath_img)
        for tag in tag_urls:
            src = tag.get_attribute('src')
            # Checking for a valid URL
            if(src is not None) and src[:4]=='http':
                image_urls.append(src)

        print(f"Available images: {len(image_urls)}")
        download_images(image_urls, keyword, n_reg)
    
    except NoSuchElementException as e:
        print(f"Error searching {keyword}: {e.msg}")
    
driver.quit()



 ****************************************
Search for fruits
Click successful
Available images: 52

 ****************************************
Search for cars
Click successful
Available images: 43

 ****************************************
Search for Machine Learning
Click successful
Available images: 26

 ****************************************
Search for Guitar
Click successful
Available images: 39

 ****************************************
Search for Cakes
Click successful
Available images: 46


4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. 
    * Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. 
    * Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV. 


In [226]:
# Importing libraries 
import selenium
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

import time
import regex as re

In [423]:
def get_attribute_el(webelement, attribute):
    return (webelement.get_attribute(attribute))

def load_items_by_xpath(driver, n=None, **kwargs):
    ''' Load items on the page with the attributes sent in kwargs.
        n: number of products required
        kwargs: dictionary with key, value = label, XPATH to retrieve 
        Return: a DataFrame with the info of the products
    '''
    data_retrieved = {}
    lenght_ref = -1
    
    for key, value in kwargs.items():
        tags = driver.find_elements(By.XPATH, value[0] )  # scraping
        
        if len(value)>1:
            items = [ value[1](tag) for tag in tags] # If lambda is specified it applies
        else:
            items = [ tag.text for tag in tags]  # Default: text function
            
        if(lenght_ref <0):
            lenght_ref = len(items)
        
        if(lenght_ref == len(items)):
            data_retrieved[key] = items
            print(f"{key} retrieved and included:", len(items))
        else:
            data_retrieved[key] = [None for _ in range(lenght_ref)]
            print(f"{key} retrieved NOT COMPATIBLE:", len(items))

    df = pd.DataFrame(data_retrieved)
        
    if ( bool(n) and (n < len(df))):
        return (df.iloc[0:n,:])
    
    return df

def remove_popup(xpath):
    # Check if an element on the new page is present
    try:
        close_button  = driver.find_element(By.XPATH, xpath)
        close_button.click() 
        print("Pop-up removed")
    except NoSuchElementException:
        print("No pop-up windows was found")

def retrieve_product_per_page(driver, xpath):
    # Retrieve the total number of products from the search and 
    # the number of products on the retrieved page
    reg = driver.find_element(By.XPATH, xpath)

    pattern = r'\b\d{1,3}(?:,\d{3})*\b|\b\d+\b'
    matches = re.findall(pattern, reg.text)
    numbers_without_commas = [match.replace(',', '') for match in matches]
    nreg = [*map(int, numbers_without_commas)]
    
    return nreg

def retrieve_products():
    ''' Retrieve the products of the page.
        Return: a dataframe with the products'''
    # Checking the found records
    try:
        first, second, total = retrieve_product_per_page(driver, '//span[@class="_10Ermr"]')
        total_records_to_retrieve = second

    except NoSuchElementException:
        print("Total records not found!")
       
    attributes = {'Smartphone name': ['//div[@class="_4rR01T"]'],
                 'Memory': ['//ul[@class="_1xgFaf"]/li[1]'],
                 'Display Size': ['//ul[@class="_1xgFaf"]/li[2]'],
                 'Cameras': ['//ul[@class="_1xgFaf"]/li[3]'],
                 'Battery Capacity': ['//ul[@class="_1xgFaf"]/li[4]'],
                 'Price': ['//div[@class="_30jeq3 _1_WHN1"]'],
                 'Product URL': ['//a[@class="_1fQZEK"]', lambda wel: get_attribute_el(wel, 'href')]}

    df_products = load_items_by_xpath(driver, None, **attributes) 
    
    print( f'Products retrieved successfully: {len(df_products)}' )
    
    # Reseting index
    df_products.reset_index(drop=True, inplace=True)
    
    return (df_products)

#### Web scraping - Begin 

In [298]:
# Set up the WebDriver 
driver = webdriver.Chrome()

url = "https://www.flipkart.com/"
driver.get(url)

# Closing pop-up window
wait_time = 5
time.sleep(wait_time)

popup_btn = '//button[@class="_2KpZ6l _2doB4z"]'
remove_popup(popup_btn)

# Search for product
time.sleep(wait_time)

product = input("Please, enter the product do you want to search? ")

xpath_input = '//input[@class="Pke_EE" or @class="_3704LK"]'
xpath_btn = '//button[@class="_2iLD__" or @class="L0Z3Pu"]'

search_product(product, xpath_input, xpath_btn)

# Web scraping the products
time.sleep(5)
df_products = retrieve_products()

# Checking the size of dataframe
print(f"Shape: {df_products.shape}")

# Cleaning driver
driver.quit()


# Recovered data
display(df_products.head())

# Cleaning the dataframe
# Colour
pattern = r'\(([^,]+),'
df_products['Colour'] = df_products['Smartphone name'].str.extract(pattern, expand=False)

# RAM/ROM
ram_pattern = r'(\d+\s+\w+)\s+RAM'
rom_pattern = r'(\d+\s+\w+)\s+ROM'
df_products['RAM'] = df_products['Memory'].str.extract(ram_pattern, expand=False)
df_products['Storage(ROM)'] = df_products['Memory'].str.extract(rom_pattern, expand=False)

# Primary Camera/Secondary Camera
rear_pattern = r'(\d+MP\s+Rear\s+Camera)'
front_pattern = r'(\d+MP\s+Front\s+Camera)'
df_products['Primary Camera'] = df_products['Cameras'].str.extract(rear_pattern, expand=False)
df_products['Secondary Camera'] = df_products['Cameras'].str.extract(front_pattern, expand=False)

brand_pattern = r'(\w+)\s'
df_products['Brand Name'] = df_products['Smartphone name'].str.extract(brand_pattern, expand=False)


df = df_products.drop(['Memory', 'Cameras'], axis=1)
df = df.fillna('-')

# Saving the data frame 
df.to_csv(f'datasets/q4_search_{product}.csv')


No pop-up windows was found
Please, enter the product do you want to search? pixel 4A
Search for pixel 4A
Click successful
Smartphone name retrieved and included: 24
Memory retrieved and included: 24
Display Size retrieved and included: 24
Cameras retrieved and included: 24
Battery Capacity retrieved and included: 24
Price retrieved and included: 24
Product URL retrieved and included: 24
Products retrieved successfully: 24
Shape: (24, 7)


,Smartphone name,Memory,Display Size,Cameras,Battery Capacity,Price,Product URL
0,"realme C53 (Champion Gold, 64 GB)",6 GB RAM | 64 GB ROM | Expandable Upto 2 TB,17.12 cm (6.74 inch) HD Display,108MP + 2MP | 8MP Front Camera,5000 mAh Battery,"₹10,999",https://www.flipkart.com/realme-c53-champion-g...
1,"Google Pixel 6a (Charcoal, 128 GB)",6 GB RAM | 128 GB ROM,15.6 cm (6.14 inch) Full HD+ Display,12.2MP + 12MP | 8MP Front Camera,4410 mAh Battery,"₹25,999",https://www.flipkart.com/google-pixel-6a-charc...
2,"Google Pixel 6a (Chalk, 128 GB)",6 GB RAM | 128 GB ROM,15.6 cm (6.14 inch) Full HD+ Display,12.2MP + 12MP | 8MP Front Camera,4410 mAh Battery,"₹25,999",https://www.flipkart.com/google-pixel-6a-chalk...
3,"realme C30 (Lake Blue, 32 GB)",2 GB RAM | 32 GB ROM | Expandable Upto 1 TB,16.51 cm (6.5 inch) HD+ Display,8MP Rear Camera | 5MP Front Camera,5000 mAh Lithium Ion Battery,"₹5,999",https://www.flipkart.com/realme-c30-lake-blue-...
4,"realme C30 (Bamboo Green, 32 GB)",3 GB RAM | 32 GB ROM | Expandable Upto 1 TB,16.51 cm (6.5 inch) HD+ Display,8MP Rear Camera | 5MP Front Camera,5000 mAh Lithium Ion Battery,"₹6,499",https://www.flipkart.com/realme-c30-bamboo-gre...


,Smartphone name,Display Size,Battery Capacity,Price,Product URL,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Brand Name
0,"realme C53 (Champion Gold, 64 GB)",17.12 cm (6.74 inch) HD Display,5000 mAh Battery,"₹10,999",https://www.flipkart.com/realme-c53-champion-g...,Champion Gold,6 GB,64 GB,-,8MP Front Camera,realme
1,"Google Pixel 6a (Charcoal, 128 GB)",15.6 cm (6.14 inch) Full HD+ Display,4410 mAh Battery,"₹25,999",https://www.flipkart.com/google-pixel-6a-charc...,Charcoal,6 GB,128 GB,-,8MP Front Camera,Google
2,"Google Pixel 6a (Chalk, 128 GB)",15.6 cm (6.14 inch) Full HD+ Display,4410 mAh Battery,"₹25,999",https://www.flipkart.com/google-pixel-6a-chalk...,Chalk,6 GB,128 GB,-,8MP Front Camera,Google
3,"realme C30 (Lake Blue, 32 GB)",16.51 cm (6.5 inch) HD+ Display,5000 mAh Lithium Ion Battery,"₹5,999",https://www.flipkart.com/realme-c30-lake-blue-...,Lake Blue,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,realme
4,"realme C30 (Bamboo Green, 32 GB)",16.51 cm (6.5 inch) HD+ Display,5000 mAh Lithium Ion Battery,"₹6,499",https://www.flipkart.com/realme-c30-bamboo-gre...,Bamboo Green,3 GB,32 GB,8MP Rear Camera,5MP Front Camera,realme
5,"realme C30 (Denim Black, 32 GB)",16.51 cm (6.5 inch) HD+ Display,5000 mAh Lithium Ion Battery,"₹5,999",https://www.flipkart.com/realme-c30-denim-blac...,Denim Black,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,realme
6,"realme C30 (Lake Blue, 32 GB)",16.51 cm (6.5 inch) HD+ Display,5000 mAh Lithium Ion Battery,"₹6,499",https://www.flipkart.com/realme-c30-lake-blue-...,Lake Blue,3 GB,32 GB,8MP Rear Camera,5MP Front Camera,realme
7,"realme C30 (Denim Black, 32 GB)",16.51 cm (6.5 inch) HD+ Display,5000 mAh Lithium Ion Battery,"₹6,499",https://www.flipkart.com/realme-c30-denim-blac...,Denim Black,3 GB,32 GB,8MP Rear Camera,5MP Front Camera,realme
8,"realme 10 Pro 5G (Nebula Blue, 128 GB)",17.07 cm (6.72 inch) Full HD+ Display,5000 mAh Battery,"₹19,999",https://www.flipkart.com/realme-10-pro-5g-nebu...,Nebula Blue,8 GB,128 GB,-,16MP Front Camera,realme
9,"realme 10 Pro 5G (Hyperspace, 128 GB)",17.07 cm (6.72 inch) Full HD+ Display,5000 mAh Battery,"₹19,999",https://www.flipkart.com/realme-10-pro-5g-hype...,Hyperspace,8 GB,128 GB,-,16MP Front Camera,realme


In [300]:
df

,Smartphone name,Display Size,Battery Capacity,Price,Product URL,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Brand Name
0,"realme C53 (Champion Gold, 64 GB)",17.12 cm (6.74 inch) HD Display,5000 mAh Battery,"₹10,999",https://www.flipkart.com/realme-c53-champion-g...,Champion Gold,6 GB,64 GB,-,8MP Front Camera,realme
1,"Google Pixel 6a (Charcoal, 128 GB)",15.6 cm (6.14 inch) Full HD+ Display,4410 mAh Battery,"₹25,999",https://www.flipkart.com/google-pixel-6a-charc...,Charcoal,6 GB,128 GB,-,8MP Front Camera,Google
2,"Google Pixel 6a (Chalk, 128 GB)",15.6 cm (6.14 inch) Full HD+ Display,4410 mAh Battery,"₹25,999",https://www.flipkart.com/google-pixel-6a-chalk...,Chalk,6 GB,128 GB,-,8MP Front Camera,Google
3,"realme C30 (Lake Blue, 32 GB)",16.51 cm (6.5 inch) HD+ Display,5000 mAh Lithium Ion Battery,"₹5,999",https://www.flipkart.com/realme-c30-lake-blue-...,Lake Blue,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,realme
4,"realme C30 (Bamboo Green, 32 GB)",16.51 cm (6.5 inch) HD+ Display,5000 mAh Lithium Ion Battery,"₹6,499",https://www.flipkart.com/realme-c30-bamboo-gre...,Bamboo Green,3 GB,32 GB,8MP Rear Camera,5MP Front Camera,realme
5,"realme C30 (Denim Black, 32 GB)",16.51 cm (6.5 inch) HD+ Display,5000 mAh Lithium Ion Battery,"₹5,999",https://www.flipkart.com/realme-c30-denim-blac...,Denim Black,2 GB,32 GB,8MP Rear Camera,5MP Front Camera,realme
6,"realme C30 (Lake Blue, 32 GB)",16.51 cm (6.5 inch) HD+ Display,5000 mAh Lithium Ion Battery,"₹6,499",https://www.flipkart.com/realme-c30-lake-blue-...,Lake Blue,3 GB,32 GB,8MP Rear Camera,5MP Front Camera,realme
7,"realme C30 (Denim Black, 32 GB)",16.51 cm (6.5 inch) HD+ Display,5000 mAh Lithium Ion Battery,"₹6,499",https://www.flipkart.com/realme-c30-denim-blac...,Denim Black,3 GB,32 GB,8MP Rear Camera,5MP Front Camera,realme
8,"realme 10 Pro 5G (Nebula Blue, 128 GB)",17.07 cm (6.72 inch) Full HD+ Display,5000 mAh Battery,"₹19,999",https://www.flipkart.com/realme-10-pro-5g-nebu...,Nebula Blue,8 GB,128 GB,-,16MP Front Camera,realme
9,"realme 10 Pro 5G (Hyperspace, 128 GB)",17.07 cm (6.72 inch) Full HD+ Display,5000 mAh Battery,"₹19,999",https://www.flipkart.com/realme-10-pro-5g-hype...,Hyperspace,8 GB,128 GB,-,16MP Front Camera,realme


In [299]:
#"Oneplus Nord, pixel 4A,"


5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps. 


In [272]:
# Importing libraries 
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

import time

In [288]:
def searching_location():
    # Set up the WebDriver 
    driver = webdriver.Chrome()

    # Open google maps
    url = 'https://www.google.com.ec/maps'
    driver.get(url)

    # Location
    location = input("Please, enter a location to search: ")
    print(f"Searching for: {location}")

    # Searching the location
    try:
        search_bar = driver.find_element(By.NAME, "q")
        search_bar.send_keys(location)
        search_bar.send_keys(Keys.RETURN)
        print('Searching...')

        # Wait for the results to load
        driver.implicitly_wait(10)

        try:
            current_url = driver.current_url
            print(f"URL Extracted: {current_url}")

            start = current_url.index('@') + 1
            end = current_url.index('?', start)
            coords = current_url[start:end].split(',')
            latitude = coords[0]
            longitude = coords[1]

            print(f'City: {location}')
            print(f"Latitude = {latitude}, Longitude = {longitude}")

        except Exception as e:
            print(f"Exception: {e.msg}")

    except NoSuchElementException as e:
        print(f"An element wasn't found: {e.msg}")

    # Close the WebDriver
    driver.quit()


In [289]:
searching_location()

Please, enter a location to search: Durán
Searching for: Durán
Searching...
URL Extracted: https://www.google.com.ec/maps/@-2.0447232,-79.9080448,15z?entry=ttu
City: Durán
Latitude = -2.0447232, Longitude = -79.9080448


In [290]:
searching_location()

Please, enter a location to search: Guayaquil
Searching for: Guayaquil
Searching...
URL Extracted: https://www.google.com.ec/maps/@-2.0447232,-79.9080448,15z?entry=ttu
City: Guayaquil
Latitude = -2.0447232, Longitude = -79.9080448


6. Write a program to scrap all the available details of best gaming laptops from digit.in. 


In [ ]:
# Importing libraries 
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException

import time

In [331]:
# Set up the WebDriver 
driver = webdriver.Chrome()

url ='https://www.digit.in/'
driver.get(url)

wait_time = 3
time.sleep(wait_time)

link = driver.find_element(By.XPATH, '//div[@class="listing_container"]//ul/li[9]')
print(f"Clicking <{link.text}>")
link.click()

# Scraping laptops
attributes = {'Laptop name': ['//span[@class="datahreflink"]/h3'],
              'Description': ['//div[@class="tptn-prod-desc"]'],
              'Operative system': ['//div[@class="Specs-Wrap"]/ul/li[1]/div/div'],
              'Display size':['//div[@class="Specs-Wrap"]/ul/li[2]/div/div'],
              'Processor':['//div[@class="Specs-Wrap"]/ul/li[4]/div/div'],
              'HD/Memory':['//div[@class="Specs-Wrap"]/ul/li[4]/div/div'],
              'Price':['//td[@class="smprice"]'],
              'Graphic processor': ['//div[@class="Spcs-details"][1]/table/tbody/tr[6]/td[3]'],
              'Body': ['//div[@class="Spcs-details"][1]/table/tbody/tr[7]/td[3]']}

time.sleep(wait_time)

df_products = load_items_by_xpath(driver, None, **attributes) 

print( f'Products retrieved successfully: {len(df_products)}' )

# Cleaning driver
driver.quit()

# Reseting index
df_products.reset_index(drop=True, inplace=True)

# Saving the dataframe
df_products.to_csv('datasets/q6_best_gaming_laptops.csv')

display (df_products)


Clicking <Best Gaming Laptops in India>
Laptop name retrieved and included: 7
Description retrieved and included: 7
Operative system retrieved and included: 7
Display size retrieved and included: 7
Processor retrieved and included: 7
HD/Memory retrieved and included: 7
Price retrieved and included: 7
Graphic processor retrieved and included: 7
Body retrieved and included: 7
Products retrieved successfully: 7


,Laptop name,Description,Operative system,Display size,Processor,HD/Memory,Price,Graphic processor,Body
0,HP OMEN 17 (2023),We tested the HP OMEN 17 2023 at our digit tes...,Windows 11 Home,"17.3"" (2560 x 1440)",13th Gen Intel Core i7-13700HX | 5.0 GHz,13th Gen Intel Core i7-13700HX | 5.0 GHz,"₹ 269,777",12 GB DDR6 NVIDIA GeForce RTX 4080 Graphics card,397.1 x 262 x 27 mm dimension & 2.78 kg weight
1,MSI Titan GT77 12UHS,This laptop won our best performance award las...,Windows 11 Home,"17.3"" (3840 x 2160)",12th Gen Intel Core i9-12900HX | 3.6 GHz,12th Gen Intel Core i9-12900HX | 3.6 GHz,"₹ 499,990",16 GB DDR6 NVIDIA GeForce RTX 3080Ti Graphics ...,397 x 330 x 23 mm dimension & 3.3 kg weight
2,Lenovo Legion 5i Pro,This Lenovo Legion 5i Pro gaming laptop hits t...,Windows 11 Home,"16"" (2560 x 1600)",12th Gen Intel Core i7-12700H | 4.7 GHz,12th Gen Intel Core i7-12700H | 4.7 GHz,"₹ 179,990",NVIDIA GeForce RTX 3070 Ti Graphics card,359.9 x 264.4 x 19.9 mm dimension & 3.6 kg weight
3,ASUS ROG Strix Scar 18 2023,If a 17-inch display on a gaming laptop no lon...,Windows 11 Home,"18"" (1920 x 1200)",13th Gen Intel Core i9-13980HX | 2.2 GHz,13th Gen Intel Core i9-13980HX | 2.2 GHz,"₹ 279,990",12 GB DDR6 NVIDIA GeForce RTX 4080 Graphics card,294 x 399 x 23 mm dimension & 3.1 kg weight
4,Acer Predator Helios Neo 16,The Acer Predator series of gaming laptops has...,Windows 11 Home,"16"" (2560 x 1600)",13th Gen Intel Core i7-13700HX | NA,13th Gen Intel Core i7-13700HX | NA,"₹ 149,990",8 GB DDR6 NVIDIA GeForce RTX 4060 Graphics card,360 x 279 x 28 mm dimension & 2.6 kg weight
5,ASUS ROG Zephyrus G14,"Another ASUS laptop makes it to our list, and ...",Windows 11 Home,"14"" (1920 x 1200)",AMD Ryzen 9-6900HS | 4.9 GHz,AMD Ryzen 9-6900HS | 4.9 GHz,"₹ 156,990",8 GB DDR6 AMD Radeon RX 6700S Graphics card,312 x 227 x 19 mm dimension & 1.65 kg weight
6,MSI Cyborg 15,One of the newest gaming laptop series from MS...,Windows 11 Home,"15.6"" (1920 x 1080)",12th Gen Intel Core i7-12650H | 4.7 GHz,12th Gen Intel Core i7-12650H | 4.7 GHz,"₹ 125,000",8 GB DDR6 NVIDIA GeForce RTX 4060 Graphics card,& 1.98 kg weight


7. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: 
“Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”. 


In [410]:
# Importing libraries 
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException

import time


In [411]:
def clic_link_withJS(xpath, elementName, js_flag=False):
    # Waiting time
    time.sleep(wait_time)

    try:
        element = driver.find_element(By.XPATH, xpath)

        # Clic the element
        if (js_flag):
            driver.execute_script("arguments[0].click();", element)
        else:
            element.click()
        print(f"{elementName} clicked")

    except NoSuchElementException:
        print(f"{elementName} wasn't find")


In [412]:
# Set up the WebDriver 
driver = webdriver.Chrome()

url ='https://www.forbes.com/'
driver.get(url)

wait_time = 5
time.sleep(wait_time)
          

try:
    xpath = '//div[@class="_69hVhdY4"]'
    clic_link_withJS(xpath, "Menu", js_flag=False)

    time.sleep(2)
    xpath = '//div[@class="mpBfVZz3"]/a'
    clic_link_withJS(xpath, "Billioners Item", js_flag=False)

    time.sleep(2)
    xpath = '//h2[contains(text(), "Billionaires List 2023")]//ancestor::a'
    clic_link_withJS(xpath, "Billioners List", js_flag=False)

except NoSuchElementException as e:
    print(f"The links can not be found: {e.msg}")

    
    
# Confirming we are in the correct page
current_url = driver.current_url
print(current_url)
if(current_url == "https://www.forbes.com/billionaires/"):

    print("Scraping...")
    # “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.
    attributes = {'Rank': ['//div[@class="TableRow_row__L-0Km"]/div[1]/div'],
                  'Name': ['//div[@class="TableRow_row__L-0Km"]/div[2]/div'],
                  'Net worth': ['//div[@class="TableRow_row__L-0Km"]/div[3]/div'],
                  'Age': ['//div[@class="TableRow_row__L-0Km"]/div[4]/div'],
                  'Citizenship': ['//div[@class="TableRow_row__L-0Km"]/div[5]'],
                  'Source': ['//div[@class="TableRow_row__L-0Km"]/div[6]/div'],
                  'Industry': ['//div[@class="TableRow_row__L-0Km"]/div[7]/div']}

    wait_time = 7
    xpath_nextbtn = '//span[@class="Pagination_bubbleArrow__WFrX4 Pagination_paginationBtnNext__IOwqm"]'
    df = pd.DataFrame()
    pag = 1
    
    while(True and pag<15):
        
        time.sleep(wait_time)

        df_pag = load_items_by_xpath(driver, None, **attributes) 
        
        # Checking there is data retrieved
        if len(df_pag) <= 0:
            print( f'Page {pag}, NO retrieved data.' )
            break
    
        # Concat the dataframes
        df = pd.concat([df,df_pag])
        print( f'Page {pag}, {len(df_pag)} records added,  Billioners retrieved successfully: {len(df)}' )
        pag += 1

        # Next page
        try:
            go_next_page(driver, xpath_nextbtn)
        except NoSuchElementException as e:
            print(f"No more data to retrieve: {e.msg}")
            break
        except ElementClickInterceptedException:
            print(f"No more data to retrieve: ElementClickInterceptedException occurred")
            break
        except ElementNotInteractableException:
            print(f"No more data to retrieve: ElementNotInteractableException occurred")
            break
        
    #Cleaning driver
    driver.quit()

    # Reseting index
    df.reset_index(drop=True, inplace=True)

    # Saving the dataframe
    df.to_csv('datasets/q7_Billioners.csv')


Menu clicked
Billioners Item clicked
Billioners List clicked
https://www.forbes.com/billionaires/
Scraping...
Rank retrieved and included: 199
Name retrieved and included: 199
Net worth retrieved and included: 199
Age retrieved and included: 199
Citizenship retrieved and included: 199
Source retrieved and included: 199
Industry retrieved and included: 199
Page 1, 199 records added,  Billioners retrieved successfully: 199
 Going to 
Rank retrieved and included: 199
Name retrieved and included: 199
Net worth retrieved and included: 199
Age retrieved and included: 199
Citizenship retrieved and included: 199
Source retrieved and included: 199
Industry retrieved and included: 199
Page 2, 199 records added,  Billioners retrieved successfully: 398
 Going to 
Rank retrieved and included: 199
Name retrieved and included: 199
Net worth retrieved and included: 199
Age retrieved and included: 199
Citizenship retrieved and included: 199
Source retrieved and included: 199
Industry retrieved and incl

In [413]:
df

,Rank,Name,Net worth,Age,Citizenship,Source,Industry
0,2,Elon Musk,$180 B,51,United States,"Tesla, SpaceX",
1,3,Jeff Bezos,$114 B,59,United States,Amazon,
2,4,Larry Ellison,$107 B,78,United States,Oracle,
3,5,Warren Buffett,$106 B,92,United States,Berkshire Hathaway,
4,6,Bill Gates,$104 B,67,United States,Microsoft,
...,...,...,...,...,...,...,...
2621,2540,Yu Rong,$1 B,51,China,Health clinics,
2622,2540,"Richard Yuengling, Jr.",$1 B,80,United States,Beer,
2623,2540,Zhang Gongyun,$1 B,60,China,Tyre manufacturing machinery,
2624,2540,Zhang Guiping & family,$1 B,71,China,Real estate,


8. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video. 

In [410]:
# Importing libraries 
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException

import time


In [427]:
# Set up the WebDriver 
driver = webdriver.Chrome()

# Specify the URL of the video
video_url = 'https://www.youtube.com/watch?v=kJQP7kiw5Fk'

# Open the video URL
driver.get(video_url)
time.sleep(10)  # Let the page load

popup_btn = '//yt-button-renderer[@id="dismiss-button"]'
remove_popup(popup_btn)

time.sleep(8)  # Let the page load

attributes = {'Comment': ['//yt-formatted-string[@id="content-text"]'],
             'Upvote': ['//span[@id="vote-count-middle"]'],
             'Time': ['//div[@id="header-author"]/yt-formatted-string/a']}

# Scroll to load more comments
for _ in range(20):
    driver.execute_script("window.scrollBy(0, 1000)")
    time.sleep(2)


df_comments = load_items_by_xpath(driver, None, **attributes) 

print( f'Comments retrieved successfully: {len(df_comments)}' )
    
# Reseting index
df_products.reset_index(drop=True, inplace=True)

# Save the DataFrame to a CSV file
df_comments.to_csv('datasets/q8_youtube_comments.csv')

df_comments

No pop-up windows was found
Comment retrieved and included: 0
Upvote retrieved and included: 0
Time retrieved and included: 0
Comments retrieved successfully: 0


,Comment,Upvote,Time


In [420]:
attributes = {'Comment': ['//yt-formatted-string[@id="content-text"]'],
             'Upvote': ['//span[@id="vote-count-middle"]'],
             'Time': ['//div[@id="header-author"]/yt-formatted-string/a']}

df_comments = load_items_by_xpath(driver, None, **attributes) 
df_comments

Comment retrieved and included: 80
Upvote retrieved and included: 80
Time retrieved and included: 80


,Comment,Upvote,Time
0,The first viewer must feel like the king of th...,418 k,hace 2 años
1,"¡No importa cuántos años pasen, esta canción p...",78,hace 9 días
2,"No matter how much time passes, this song will...",66,hace 1 día (editado)
3,6 Years and this song never gets old.It's just...,111,hace 2 días
4,28/07/2023. Super Nostalgia + Tbt de Respeito ...,106,hace 8 días
...,...,...,...
75,Igual de vacana!,1,hace 9 horas
76,2023 y todavia escuchando este temazo,2,hace 4 días
77,6 años de esta joyita,1,hace 5 horas
78,All of us here for checking the number of view...,140,hace 4 años


In [426]:
# Close the driver
driver.quit()